In [11]:
import numpy as np
import os
from nltk.stem import PorterStemmer
import nltk
import enchant
import pickle

# PREPROCESS DATA

In [12]:
def preprocess(content,U,totalCount):
    d={}
    words=content.split()
    ps = PorterStemmer()
    engDict=enchant.Dict("en_US")
    for word in words:
        
        if (not word.isalnum()) and (word != '$'):
            continue
        if word.isnumeric():
            value="NUMBER"
        else:
            value=word.lower()
            if engDict.check(value):
                value=ps.stem(value)
            else:
                value="UNKNOWN"
        
        if(d.get(value) == None):
            if U.get(value) == None:
                U[value]=totalCount
                totalCount+=1
            d[value]=1
        else:
            d[value]=d[value]+1
            
    
    
    return d

def readData(PATH,spam,U,totalCount):
    dirList=os.listdir(PATH)
    size=len(dirList)
#     print(size)
    X=[]
    y=[spam for i in range(size)]
    c=0
    for file in dirList:
        with open(PATH+'/'+file, 'r',errors='ignore') as f:
#             print(file)
            content=f.read()
            contentDict=preprocess(content,U,totalCount)
#         print(contentDict)
        X.append(contentDict)
#         if c==10:
#             break
#         c+=1
    return X,y

def makeDataStructure(spamX,spamY,U):
    m=len(spamX)
    d=len(U)
    X=np.zeros([d,m])
    for i in range(m):
        for word in spamX[i]:
            if U.get(word) == None:
                continue
            X[U[word]][i]=spamX[i][word]

    return X
    
        

U={}   
totalCount=0
SPAM_PATH="Dataset/enron1/spam"
HAM_PATH="Dataset/enron1/ham"
spamX,spamY=readData(SPAM_PATH,1,U,totalCount)
hamX,hamY=readData(HAM_PATH,0,U,totalCount)
# print(U)

# print(X)
# listOfWords=list(U)


KeyboardInterrupt: 

In [ ]:
XSpam=makeDataStructure(spamX,spamY,U)
XHam=makeDataStructure(hamX,hamY,U)

In [ ]:
XSpam

In [ ]:
def storeData(XSpam,XHam,U):
    spamfile=open('spam', 'wb')
    hamfile=open('ham', 'wb')
    Ufile= open('u', 'wb')
    
    
    pickle.dump(XSpam,spamfile)
    pickle.dump(XHam,hamfile)
    pickle.dump(U, Ufile)
    
    spamfile.close()
    hamfile.close()
    Ufile.close()
    
# storeData(XSpam,XHam,U)

# TRAINING

In [ ]:
import numpy as np
import os
from nltk.stem import PorterStemmer
import nltk
import enchant
import pickle

In [ ]:
def loadData():
    spamfile=open('spam', 'rb')
    hamfile=open('ham', 'rb')
    Ufile= open('u', 'rb')
    
    U=pickle.load(Ufile)
    XSpam=pickle.load(spamfile)
    XHam=pickle.load(hamfile)
    
    spamfile.close()
    hamfile.close()
    Ufile.close()
    
    return U,XSpam,XHam

U,XSpam,XHam=loadData()

In [ ]:
len(U)

In [ ]:
def storeRepresentation(p,mean):
    pfile = open('p', 'wb')
    meanfile = open('mean', 'wb')
    
    pickle.dump(p, pfile)
    pickle.dump(mean, meanfile)
    
    pfile.close()
    meanfile.close()
    
def loadReprsentation():
    pfile = open('p-f', 'rb')
    meanfile = open('mean-f', 'rb')
    Ufile= open('u', 'rb')
    
    p=pickle.load(pfile)
    mean=pickle.load(meanfile)
    U=pickle.load(Ufile)
    
    pfile.close()
    meanfile.close()
    Ufile.close()
    
    return p,mean,U

In [ ]:
# np.concatenate((XHam,XSpam),axis=1)

In [ ]:
def naiveBayesTraining(spamX,hamX):
    spamSize=np.sum(np.sum(spamX))
    hamSize=np.sum(np.sum(hamX))
    size=hamX.shape[1]+spamX.shape[1]
    mean=np.zeros([spamX.shape[0],2])
    p=spamSize/(spamSize+hamSize)
    mean[:,1]=np.sum(spamX,axis=1)/spamSize
    mean[:,0]=np.sum(hamX,axis=1)/hamSize

    
    return p,mean

d=XSpam.shape[0]
# ones=np.ones([d,1])


# laplace smoothing
# XSpam=np.concatenate((XSpam,ones),axis=1)
# XHam=np.concatenate((XHam,ones),axis=1)

# training
p,mean=naiveBayesTraining(XSpam,XHam)
print(mean.shape)

In [7]:
# XSpam=np.concatenate((XSpam,ones),axis=1)

In [8]:
#store parameters
storeRepresentation(p,mean)

# TESTING

In [13]:
import numpy as np
import os
from nltk.stem import PorterStemmer
import nltk
import enchant
import pickle

In [14]:
def probPart(x,mean):
    d=x.shape[0]
#     print(mean.shape)
    prob=1
    count=10000000000
    for i in range(d):
        if x[i] != 0 :
            if mean[i] != 0:
                prob*=x[i]*mean[i]
            else:
                prob*=(1/count)
                
        else:
            if mean[i] != 1:
                prob*=(1-mean[i])
            else:
                prob*=1/count
                
    return prob
            
    
def naiveBayesPrediction(x,p,mean):
    spamProbability=probPart(x,mean[:,1])*p
    hamProbability=probPart(x,mean[:,0])*(1-p)
    if(spamProbability >= hamProbability):
        return 1
    else:
        return 0
    

In [15]:
def accuracyMeasure():
    pass

In [16]:
def loadReprsentation():
    pfile = open('p', 'rb')
    meanfile = open('mean', 'rb')
    Ufile= open('u', 'rb')
    
    p=pickle.load(pfile)
    mean=pickle.load(meanfile)
    U=pickle.load(Ufile)
    
    pfile.close()
    meanfile.close()
    Ufile.close()
    
    return p,mean,U

def testPreprocess(content,U):
    words=content.split()
    ps = PorterStemmer()
    engDict=enchant.Dict("en_US")
    d=len(U)
    xTest=np.zeros([d,1])
    for word in words:
        if (not word.isalnum()) and (word != '$'):
            continue
        if word.isnumeric():
            value="NUMBER"
        else:
            value=word.lower()
            if engDict.check(value):
                value=ps.stem(value)
            else:
                value="UNKNOWN"
        
        if(U.get(value) == None):
            continue
            
        xTest[U[value]][0]+=1
                
    return xTest

def prediction(file,U,p,mean):
    content=file.read()
    xTest=testPreprocess(content,U)
#     print(xTest.shape)
#     print(mean.shape)
    yPredicted=naiveBayesPrediction(xTest,p,mean)
    print(yPredicted)

# with open("Dataset/enron1/spam/0006.2003-12-18.GP.spam.txt") as f:
#     p,mean,U=loadReprsentation()
#     prediction(f,U,p,mean)
    

In [18]:
#testing folder
TEST_PATH_SPAM="Dataset/enron6/spam"
TEST_PATH_HAM="Dataset/enron6/ham"
p,mean,U=loadReprsentation()

# print(mean)
def test(PATH,p,mean,U,spam):
    
    dirList=os.listdir(PATH)
    size=len(dirList)
    count=0
    c=0
    for file in dirList:
        with open(PATH+'/'+file, 'r',errors='ignore') as f:
                content=f.read()
                xTest=testPreprocess(content,U)
                yPredicted=naiveBayesPrediction(xTest,p,mean)
                count+=yPredicted
                
#         if c==100:
#             break
#         c+=1
#     size=c
#     if spam == 1:
#         print(count/size)
#     else:
#         print(1 - count/size)
    return count,size
                
count1,size1=test(TEST_PATH_SPAM,p,mean,U,1)
count2,size2=test(TEST_PATH_HAM,p,mean,U,0) 

print("spam accuracy :- ",count1/size1)
print("ham accuracy :- ",(size2-count2)/size2)
print("Overall accuracy :- ",(count1 + size2-count2)/(size1+size2))
    

spam accuracy :-  0.9817777777777777
ham accuracy :-  0.568
Overall accuracy :-  0.8783333333333333


In [27]:
def predictOnTestData(PATH):
    p,mean,U=loadReprsentation()
    dirList=os.listdir(PATH)
    size=len(dirList)
    yPredictedDict={}
    for file in dirList:
        with open(PATH+'/'+file, 'r',errors='ignore') as f:
                content=f.read()
                xTest=testPreprocess(content,U)
                yPredicted=naiveBayesPrediction(xTest,p,mean)
                yPredictedDict[file]=yPredicted
    
    print(yPredictedDict)
    
PATH="test/"
predictOnTestData(PATH)
    
    
        
    

{'ham2.txt': 1, 'ham1.txt': 1, 'spam1.txt': 1, 'spam3.txt': 1}
